# Import packages

In [1]:
#!pip install docplex
#!pip install folium

import pandas as pd
import numpy as np
import math 
import folium
from datetime import date
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

# Read and process data

In [2]:
# Read historical Covid-19-data
iva_df = pd.read_csv("./data/iva_kumulativ.csv",sep=";")
iva_df.Region = iva_df.Region.apply(lambda x: x.replace("Region ", ""))
iva_df = iva_df.set_index(iva_df.Region)
iva_df = iva_df.drop(["Region"],axis=1)
iva_df = iva_df.drop('Hela riket')

# Process historical data
data_df = pd.DataFrame(columns=["Date","Region","IVA"])
for ind, row in iva_df.iterrows(): 
    temp = pd.DataFrame(data = row,index= row.index)
    temp.columns = ["IVA"]
    temp["Region"] = ind
    temp["Date"] = temp.index
    temp = temp.reset_index(drop=True)
    data_df = data_df.append(temp)
data_df = data_df.reset_index(drop=True)

# Read region data
region_df = pd.read_csv("./data/regions.csv",sep=";")
region_df.Region = region_df.Region.apply(lambda x: x.replace("Region ", ""))
    
# Extract current still image
#today = date.today()
today = date(2020,3,25)
target_day = today + timedelta(days=7)
current_df = data_df[data_df["Date"] == today.strftime("%Y-%m-%d")]

# Merge current_df with region data
current_df = current_df.merge(region_df, on=['Region'], how='left')

# Process current_df
current_df["UnderCapacity"] = current_df["IVA"] - current_df["Capacity"]
current_df["UnderCapacity"] = current_df["UnderCapacity"].apply(lambda x: max(0,x))
current_df["SurplusCapacity"] = current_df["Capacity"] - current_df["IVA"]
current_df["SurplusCapacity"] = current_df["SurplusCapacity"].apply(lambda x: max(0,x))
current_df = current_df.set_index(current_df.Region)

In [3]:
region_data = {}
for d in list(region_df.Region):
    data = data_df[data_df.Region==d]
    data = data.groupby('Date').sum().reset_index()
    data['Date']=pd.to_datetime(data['Date'])
    data = data.sort_values(by=['Date'], ascending=False)
    region_data[d] = data  

# Visualize data

In [4]:
map_pre = folium.Map(location=[62.212927,15.134684], zoom_start=5)

for index, v in current_df.iterrows():  
    if v.Capacity > v.IVA + 5:
        color = 'blue'
    elif v.Capacity > v.IVA:
        color='green'
    elif v.Capacity == v.IVA:
        color='orange'
    elif v.Capacity < v.IVA:
        color='red'

    folium.Circle(location=[v.Lat, v.Long],
                  radius=5000 + v.IVA*200,
                  popup=folium.Popup(html = v.Region + '<br>'+str(v.IVA)+' cases<br>'+ str(v.Capacity) + ' max',
                                     max_width=250,min_width=50),
                  fill=True,color=color).add_to(map_pre)
map_pre

# Optimization
## Check environment
If CPLEX library isn't present... then you've got some installation stuff to do. I'll probably implement this solution later on in PuLP with an open-source solver. Hang on until then. 

In [5]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Windows 64bit
* Python version 3.7.4, located at: C:\Users\tilda.lundgren\AppData\Local\Continuum\anaconda3\python.exe
* docplex is present, version is (2, 10, 155)
* CPLEX library is present, version is 12.9.0.0, located at: C:\Users\tilda.lundgren\AppData\Local\Continuum\anaconda3\lib\site-packages
* pandas is present, version is 0.25.1


## Set up model, parameters, sets and variables

In [6]:
# Model 
from docplex.mp.model import Model
mdl = Model("PatientAllocations")

# Parameters
NB_PERIODS = 1
MAX_NB_LONG_TRANSFERS_PER_PERIOD = 1000
MAX_CASES_PER_LONG_TRANSFERS = 1000
MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT = 1000
THRESHOLD_FOR_LONG_DISTANCE = 200

# Sets
mdl.deps = list(region_df.Region)
mdl.transfer_periods = list(range(NB_PERIODS))
mdl.all_periods = list(range(NB_PERIODS+1))

# Variables
mdl.x_vars = {(d1,d2,p): mdl.binary_var(name="x_{0}_{1}_{2}".format(d1,d2,p)) 
              for d1 in mdl.deps for d2 in mdl.deps for p in mdl.transfer_periods}
mdl.y_vars = {(d1,d2,p): mdl.integer_var(name="y_{0}_{1}_{2}".format(d1,d2,p), ub = MAX_CASES_PER_LONG_TRANSFERS) 
              for d1 in mdl.deps for d2 in mdl.deps for p in mdl.transfer_periods}
mdl.o_vars = {(d,p): mdl.integer_var(name="o_{0}_{1}".format(d,p), lb = 0) 
              for d in mdl.deps for p in mdl.all_periods}
mdl.max_under = mdl.continuous_var(lb=0, name = "max_under")
mdl.max_over = mdl.continuous_var(lb=0, name = "max_over")

## Calculate distances

In [7]:
def distance(lt1, lg1, lt2, lg2):
    R = 6373.0
    lat1 = math.radians(lt1); lon1 = math.radians(lg1);
    lat2 = math.radians(lt2); lon2 = math.radians(lg2);
    dlon = lon2 - lon1; dlat = lat2 - lat1;
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

dep_distances = {d1:{d2: distance(region_df[region_df.Region == d1].Lat.values[0], 
                            region_df[region_df.Region == d1].Long.values[0], 
                            region_df[region_df.Region == d2].Lat.values[0], 
                            region_df[region_df.Region == d2].Long.values[0])
               for d2 in mdl.deps} 
           for d1 in mdl.deps}

is_long = {d1:{d2: distance(region_df[region_df.Region == d1].Lat.values[0], 
                            region_df[region_df.Region == d1].Long.values[0], 
                            region_df[region_df.Region == d2].Lat.values[0], 
                            region_df[region_df.Region == d2].Long.values[0]) > THRESHOLD_FOR_LONG_DISTANCE
               for d2 in mdl.deps} 
           for d1 in mdl.deps}

## Define constraints

In [8]:
# ---------------------------------------------------------------------------------- #
# Exempt regions (will not recieve not patients)
# ---------------------------------------------------------------------------------- #
exempt = ["Gotland","Jämtland Härjedalen"]

for d in exempt: 
    for t in mdl.transfer_periods:
        mdl.add_constraint(ct = mdl.sum(mdl.y_vars[dx, d, t] for dx in mdl.deps) == 0, 
                           ctname = "exempt_regions_{0}_{1}".format(d, t))

# ---------------------------------------------------------------------------------- #            
# Set initial state
# ---------------------------------------------------------------------------------- #
for d in mdl.deps: 
    mdl.add_constraint(ct = mdl.o_vars[d, 0] == current_df.at[d,"IVA"],
                       ctname = "initial_state_{0}".format(d))
    

# ---------------------------------------------------------------------------------- #
# Structural constraint between x_vars and y_vars
# ---------------------------------------------------------------------------------- #
for pair in ((d1, d2) for d1 in mdl.deps for d2 in mdl.deps):
    for t in mdl.transfer_periods:
        mdl.add_constraint(ct = mdl.x_vars[pair[0], pair[1], t] == (mdl.y_vars[pair[0], pair[1], t] >= 1) , 
                           ctname = "use_link_{0}_{1}_{2}".format(pair[0], pair[1], t))

# ---------------------------------------------------------------------------------- #
# Short transfers bounds
# ---------------------------------------------------------------------------------- #
for pair in ((d1, d2) for d1 in mdl.deps for d2 in mdl.deps):
    if not is_long[pair[0]][pair[1]]:
        for t in mdl.transfer_periods:
            mdl.add_constraint(ct = mdl.y_vars[pair[0], pair[1], t] <= 1, 
                               ctname = "short_transfer_bound_{0}_{1}_{2}".format(pair[0], pair[1], t))

# ---------------------------------------------------------------------------------- #
# Number of transfers from a department less than current number of cases
# ---------------------------------------------------------------------------------- #
for d in mdl.deps: 
    for t in mdl.transfer_periods:
        mdl.add_constraint(ct = mdl.sum(mdl.y_vars[d, dx, t] for dx in mdl.deps) <= mdl.o_vars[d,t],
                           ctname = "transfer_less_than_current_cases_{0}_{1}".format(d,t))

    
# ---------------------------------------------------------------------------------- #
# Maximum number of LONG transfers per period
# ---------------------------------------------------------------------------------- #        
for t in mdl.transfer_periods:
    long_transfers = mdl.sum(mdl.x_vars[d1, d2, t] for d1 in mdl.deps for d2 in mdl.deps if is_long[d1][d2])
    mdl.add_constraint(ct = long_transfers <= MAX_NB_LONG_TRANSFERS_PER_PERIOD,
                       ctname = "max_long_transfers_{0}".format(t))
    

# ---------------------------------------------------------------------------------- #
# Maximum number of SHORT transfers per department
# ---------------------------------------------------------------------------------- #  
for d in mdl.deps:
    short_transfers = mdl.sum(mdl.x_vars[dx, d, t] for dx in mdl.deps if not is_long[dx][d] for t in mdl.transfer_periods)
    mdl.add_constraint(ct = short_transfers <= MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT,
                       ctname = "max_short_transfers_{0}".format(d))
    

# ---------------------------------------------------------------------------------- #
# Update number of cases for next period
# ---------------------------------------------------------------------------------- #     
for d in mdl.deps:
    for t in mdl.transfer_periods:
        
        # Calculate existing cases and reallocations
        existing_cases = mdl.o_vars[d, t] 
        cases_in = mdl.sum(mdl.y_vars[dx, d, t] for dx in mdl.deps)
        cases_out = mdl.sum(mdl.y_vars[d, dx, t] for dx in mdl.deps)
      
        # Get prognosis for organic growth
        target_day = today + timedelta(days=7)
        prognosis = region_data[d][region_data[d]["Date"] == target_day].IVA.values[0]
        current = region_data[d][region_data[d]["Date"] == today].IVA.values[0]
        organic_growth = prognosis - current #Prediction for organic growth goes here 
        
        #Summarize new cases
        new_cases = existing_cases + cases_in - cases_out + organic_growth
        
        # Add constraint
        mdl.add_constraint(ct = mdl.o_vars[d, t+1] == new_cases,
                           ctname = "new_cases_{0}_{1}".format(d,t))


## Define objective

In [9]:
# ---------------------------------------------------------------------------------- #
# Total undercapacity
# ---------------------------------------------------------------------------------- #  
total_undercapacity = mdl.sum(mdl.max(0,mdl.o_vars[d, NB_PERIODS] - current_df.at[d,"Capacity"] ) 
                              for d in mdl.deps )

# ---------------------------------------------------------------------------------- #
# Even distribution of undercapacity
# ---------------------------------------------------------------------------------- #   
for d in mdl.deps:
    under_capacity = mdl.max(0,mdl.o_vars[d, NB_PERIODS] - current_df.at[d,"Capacity"] )
    mdl.add_constraint(ct = mdl.max_under >= under_capacity,
                       ctname = "max_under_capacity_{0}".format(d))

# ---------------------------------------------------------------------------------- #
# Even distribution of overcapacity
# ---------------------------------------------------------------------------------- #   
for d in mdl.deps:
    over_capacity = mdl.max(0, current_df.at[d,"Capacity"] - mdl.o_vars[d, NB_PERIODS]) 
    mdl.add_constraint(ct = mdl.max_over >= over_capacity,
                       ctname = "max_over_capacity_{0}".format(d))
    
# ---------------------------------------------------------------------------------- #
# Minimize patient transfers
# ---------------------------------------------------------------------------------- #   
nb_long_transfers = mdl.sum(mdl.x_vars[d1, d2, t] 
                            for d1 in mdl.deps 
                            for d2 in mdl.deps 
                            if is_long[d1][d2] 
                            for t in mdl.transfer_periods)

nb_short_transfers = mdl.sum(mdl.x_vars[d1, d2, t] 
                             for d1 in mdl.deps 
                             for d2 in mdl.deps 
                             if not is_long[d1][d2] 
                             for t in mdl.transfer_periods)

nb_patient_transfers = mdl.sum(mdl.y_vars[d1, d2, t] 
                             for d1 in mdl.deps 
                             for d2 in mdl.deps
                             for t in mdl.transfer_periods)

km_patient_transfers = mdl.sum(mdl.y_vars[d1, d2, t] * dep_distances[d1][d2]
                             for d1 in mdl.deps 
                             for d2 in mdl.deps
                             for t in mdl.transfer_periods)

# ---------------------------------------------------------------------------------- #
# Summarize all
# ---------------------------------------------------------------------------------- #   
mdl.minimize(100   * total_undercapacity +\
             100   * mdl.max_under + \
             1     * mdl.max_over + \
             1     * nb_patient_transfers + \
             0.1   * km_patient_transfers + \
             1     * nb_long_transfers)

#mdl.minimize(total_undercapacity)

## Solve model

In [10]:
mdl.set_time_limit(60); #Seconds
mdl.parameters.mip.strategy.probe.set(0);
mdl.parameters.parallel.set(-1); #  opportunistic parallel search mode
mdl.parameters.threads.set(4);

mdl.solve(log_output=True,lex_mipgaps = [0.001])

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_Parallel                                -1
CPXPARAM_TimeLimit                               60
CPXPARAM_MIP_Tolerances_MIPGap                   0.001
Tried aggregator 2 times.
MIP Presolve eliminated 523 rows and 537 columns.
Aggregator did 571 substitutions.
Reduced MIP has 546 rows, 688 columns, and 1962 nonzeros.
Reduced MIP has 345 binaries, 227 generals, 0 SOSs, and 194 indicators.
Presolve time = 0.03 sec. (3.21 ticks)
Found incumbent of value 8345.000000 after 0.03 sec. (3.85 ticks)
Probing fixed 0 vars, tightened 114 bounds.
Probing time = 0.00 sec. (2.27 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
Reduced MIP has 560 rows, 688 columns, and 1990 nonzeros.
Reduced MIP has 345 binaries, 227 generals, 0 SOSs, and 179 indicators.
Presolve time = 0.02 sec. (1.45 ticks)
Probing time = 0.02 sec. (1.92 ticks)
MIP emphasis: balance optimality a

docplex.mp.solution.SolveSolution(obj=2405.09,values={x_Norrbotten_Väste..

In [11]:
print("Total undercapacity:", total_undercapacity.solution_value, "patients")
print("Max region undercapacity:", mdl.max_under.solution_value, "patients")
print("Max region overcapacity: ",mdl.max_over.solution_value,"patients")
print("Patient transfers:",nb_patient_transfers.solution_value,"patients")
print("Transfer kilometers:", round(km_patient_transfers.solution_value), "km")
print("Long transfers:",nb_long_transfers.solution_value)
print("Short transfers:",nb_short_transfers.solution_value)

Total undercapacity: 6.0 patients
Max region undercapacity: 6.0 patients
Max region overcapacity:  41.0 patients
Patient transfers: 49 patients
Transfer kilometers: 11071 km
Long transfers: 8
Short transfers: 13


# Interpret solution

## Process solution

In [12]:
edges = [(t, d1, d2, int(mdl.y_vars[d1, d2, t]), is_long[d1][d2]) 
         for t in mdl.transfer_periods 
         for d1 in mdl.deps 
         for d2 in mdl.deps 
         if int(mdl.y_vars[d1, d2, t]) >= 1]
print(edges)
print("\nNumber of transfers:",len(edges))

[(0, 'Norrbotten', 'Västerbotten', 1, True), (0, 'Stockholm', 'Dalarna', 3, True), (0, 'Stockholm', 'Gävleborg', 1, False), (0, 'Stockholm', 'Jönköpings län', 3, True), (0, 'Stockholm', 'Kalmar län', 5, True), (0, 'Stockholm', 'Uppsala', 1, False), (0, 'Stockholm', 'Värmland', 8, True), (0, 'Stockholm', 'Örebro län', 1, False), (0, 'Stockholm', 'Östergötland', 1, False), (0, 'Sörmland', 'Gävleborg', 1, False), (0, 'Sörmland', 'Jönköpings län', 1, False), (0, 'Sörmland', 'Kalmar län', 4, True), (0, 'Sörmland', 'Kronoberg', 8, True), (0, 'Sörmland', 'Uppsala', 1, False), (0, 'Sörmland', 'Värmland', 1, False), (0, 'Sörmland', 'Örebro län', 1, False), (0, 'Sörmland', 'Östergötland', 1, False), (0, 'Sörmland', 'Västra Götalandsregionen', 4, True), (0, 'Västmanland', 'Gävleborg', 1, False), (0, 'Västmanland', 'Uppsala', 1, False), (0, 'Västmanland', 'Örebro län', 1, False)]

Number of transfers: 21


In [13]:
final = {d: int(mdl.o_vars[d, NB_PERIODS].solution_value) for d in mdl.deps}

organic = {}
for d in mdl.deps:    
    prognosis = region_data[d][region_data[d]["Date"] == target_day].IVA.values[0]
    current = region_data[d][region_data[d]["Date"] == today].IVA.values[0]
    organic_growth = prognosis - current
    organic[d] = organic_growth
        
current_df["Final"] = [final[d] for d in mdl.deps]
current_df["OrganicGrowth"] = [organic[d] for d in mdl.deps]
current_df["FinalUnderCapacity"] = current_df["Final"] - current_df["Capacity"]
current_df["FinalUnderCapacity"] = current_df["FinalUnderCapacity"].apply(lambda x: max(0,x))
current_df["FinalSurplusCapacity"] = current_df["Capacity"] - current_df["Final"]
current_df["FinalSurplusCapacity"] = current_df["FinalSurplusCapacity"].apply(lambda x: max(0,x))
current_df["Allocation"] = current_df["Final"] - current_df["IVA"] - current_df["OrganicGrowth"] 

total_undercapacity_final = current_df['FinalUnderCapacity'].sum()
total_undercapacity_before = current_df['UnderCapacity'].sum()
print("Total undercapacity before reallocation:",total_undercapacity_before)
print("Total undercapacity after reallocation:",total_undercapacity_final)

Total undercapacity before reallocation: 8
Total undercapacity after reallocation: 6


In [14]:
from folium_scripts import get_arrows, get_bearing

map_fin = folium.Map(location=[62.212927,15.134684], zoom_start=5)

for index, v in current_df.iterrows():  
    
    # Choose color for circle marker
    if v.Capacity > v.Final + 5:
        color = 'blue'
    elif v.Capacity > v.Final:
        color='green'
    elif v.Capacity == v.Final:
        color='orange'
    elif v.Capacity < v.Final:
        color='red'

    # Draw circle marker
    folium.Circle(location=[v.Lat, v.Long],
                  radius=5000 + v.IVA*500,
                  popup=folium.Popup(html = v.Region + \
                                            '<br>Before: '+str(v.IVA)+\
                                            '<br>After: '+str(v.Final)+\
                                            '<br>Organic: '+str(v.OrganicGrowth)+\
                                            '<br>Allocation: '+str(v.Allocation)+\
                                            '<br>Capacity: '+str(v.Capacity),
                                     max_width=250,min_width=50),
                  fill=True,color=color).add_to(map_fin)
    
    # Draw planned transportation lines
    for (period, d1, d2, nb, il) in edges: 
        coordinates = [[current_df.at[d1,"Lat"], current_df.at[d1,"Long"]], 
                       [current_df.at[d2,"Lat"], current_df.at[d2,"Long"]]]
        if not il:
            color = 'black'; weight = 2; 
        else:
            color = 'blue'; weight = 2;
        
        pl = folium.PolyLine(coordinates, color=color, weight=weight)
        map_fin.add_child(pl)
        
        #arrows = get_arrows(locations=coordinates, color=color, size=4, n_arrows=1)
        #for arrow in arrows:
        #    arrow.add_to(map_fin)

In [15]:
map_fin